In [17]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [81]:
import os
import itertools
import codecs
import re
import datetime
# import cairocffi as cairo
# import editdistance
import numpy as np
from scipy import ndimage
import pylab
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks

## Model

In [122]:
run_name, start_epoch, stop_epoch = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S'), 0, 10

# Input Parameters
img_h = 48
img_w = 1000
words_per_epoch = 1000
val_split = 0.1
val_words = int(words_per_epoch * (val_split))
print(val_words)
# Network parameters
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 512
minibatch_size = 32

input_shape = (img_w, img_h, 1)


100


In [94]:
act = 'relu'
input_data = Input(name='the_input', shape=input_shape, dtype='float32')
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv1')(input_data)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv2')(inner)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

conv_to_rnn_dims = (img_w // (pool_size ** 2),
                    (img_h // (pool_size ** 2)) * conv_filters)
inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

# cuts down input size going into RNN:
inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

# Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = GRU(rnn_size, return_sequences=True,
            kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = GRU(rnn_size, return_sequences=True,
             go_backwards=True, kernel_initializer='he_normal',
             name='gru1_b')(inner)
gru1_merged = add([gru_1, gru_1b])
gru_2 = GRU(rnn_size, return_sequences=True,
            kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True,
             kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

# transforms RNN output to character activations:
inner = Dense(128, kernel_initializer='he_normal',
              name='dense2')(concatenate([gru_2, gru_2b]))
y_pred = Activation('softmax', name='softmax')(inner)
Model(inputs=input_data, outputs=y_pred).summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 1000, 48, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 1000, 48, 16) 160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 500, 24, 16)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 500, 24, 16)  2320        max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [104]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [105]:
labels = Input(name='the_labels',
               shape=[128], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
# Keras doesn't currently support loss funcs with extra parameters
# so CTC loss is implemented in a lambda layer
loss_out = Lambda(
    ctc_lambda_func, output_shape=(1,),
    name='ctc')([y_pred, labels, input_length, label_length])

In [106]:
# clipnorm seems to speeds up convergence
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

model = Model(inputs=[input_data, labels, input_length, label_length],
              outputs=loss_out)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)

In [111]:
# captures output of softmax so we can decode the output during visualization
test_func = K.function([input_data], [y_pred])

In [163]:
model.input

[<tf.Tensor 'the_input_8:0' shape=(?, 1000, 48, 1) dtype=float32>,
 <tf.Tensor 'the_labels_6:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'input_length_6:0' shape=(?, 1) dtype=int64>,
 <tf.Tensor 'label_length_6:0' shape=(?, 1) dtype=int64>]

In [161]:
gen = dataGenerator(data_container.training_set, 128)

model.fit_generator(
    generator=gen,
    steps_per_epoch=128,
    epochs=1)
#     validation_data=data_container.test_set,
#     validation_steps=val_words // minibatch_size,
#     callbacks=[viz_cb, img_gen],
#     initial_epoch=start_epoch)

Epoch 1/1


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 1 arrays: [array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0....

In [160]:
def dataGenerator(pathes, batch_size):

    while True: #generators for keras must be infinite
        for path in pathes:
            x_train, y_train = path

            totalSamps = x_train.shape[0]
            batches = totalSamps // batch_size

            if totalSamps % batch_size > 0:
                batches+=1

            for batch in range(batches):
                section = slice(batch*batch_size,(batch+1)*batch_size)
                yield (x_train[section], y_train[section])

## Data

In [1]:
import os
from library import lineest, util
from collections import Counter
import click
from PIL import Image
import numpy as np
import bidi.algorithm as bd

In [2]:
class GroundTruthContainer(object):
    """
    Container for ground truth used during training.
    Attributes:
        training_set (list): List of tuples (image, text) for training
        test_set (list): List of tuples (image, text) for testing
        alphabet (str): Sorted string of all codepoint found in the ground
                        truth
    """
    def __init__(self, images=None, split=lambda x: os.path.splitext(x)[0],
                 suffix='.gt.txt', normalization=None, reorder=True,
                 partition=0.9, pad=16, directory=''):
        """
        Reads a list of image-text pairs and creates a ground truth set.
        Args:
            images (list): List of file paths of line images
            split (func): Function for generating the base name without
                          extensions from paths
            suffix (str): Suffix to attach to image base name for text
                          retrieval
            normalization (str): Unicode normalization for gt
            reorder (bool): Whether to rearrange code points in "display"/LTR
                            order
            partition (float): Ground truth data partition ratio between
                               train/test set.
            pad (int): Padding to add to images left and right
        """
        self.lnorm = lineest.CenterNormalizer()
        self.training_set = []
        self.test_set = []
        self.training_alphabet = Counter()
        self.test_alphabet = Counter()

        if not images:
            return
        for line in images:
            self.add(line, split, suffix, normalization, reorder, pad, directory=directory)

        self.repartition(partition)

    def add(self, image, split=lambda x: os.path.splitext(x)[0],
                 suffix='.gt.txt', normalization=None, reorder=True,
                 pad=16, directory=''):
        """
        Adds a single image to the training set.
        """
        with click.open_file(directory + split(image) + suffix, 'r', encoding='utf-8') as fp:
            gt = fp.read()
            if normalization:
                gt = unicodedata.normalize(normalization, gt)
            if reorder:
                gt = bd.get_display(gt)

            im = Image.open(image)
            im = util.dewarp(self.lnorm, im)
            im = util.pil2array(im)
            im = lineest.prepare_line(im, pad)
            self.training_set.append((im, gt))

    def repartition(self, partition=0.9):
        """
        Repartitions the training/test sets.
        Args:
            partition (float): Ground truth data partition ratio between
                               training/test sets.
        """
        self.training_set = self.training_set + self.test_set
        idx = np.random.choice(len(self.training_set), int(len(self.training_set) * partition), replace=False)
        tmp_set = [self.training_set[x] for x in idx]
        [self.training_set.pop(x) for x in sorted(idx, reverse=True)]
        self.test_set = self.training_set
        self.training_set = tmp_set

        self.training_alphabet = Counter(''.join(t for _, t in self.training_set))
        self.test_alphabet = Counter(''.join(t for _, t in self.test_set))

    def sample(self):
        """
        Samples a line image-text pair from the training set.
        Returns:
            A tuple (line, text) with line being a numpy.array run through
            kraken.lib.lstm.prepare_line.
        """
        return self.training_set[np.random.choice(len(self.training_set))]

In [80]:
dataset = np.genfromtxt('data/manifest.txt', delimiter='\n', dtype=str)
print(dataset.shape)
data_container = GroundTruthContainer(images=list(dataset), directory='data/')

(790,)


/anaconda3/envs/muteferrika/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:450: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


In [86]:
shapes = [data_container.training_set[sh][1] for sh in range(len(data_container.training_set))]

In [92]:
max([len(list(i))for i in shapes])

78

In [146]:
training_X = np.asarray([data_container.training_set[i][0] for i in range(len(data_container.training_set))])
training_y = np.asarray([list(data_container.training_set[i][1]) for i in range(len(data_container.training_set))])

In [148]:
# training_y